In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd
# Set the search parameters
# Make sure to replace 'music1.xlsx' with actual name of your file in your directory
filename = 'Churn_Modelling.csv'
search_path = '/content/drive/MyDrive'
# Search for the file
file_path = None
for root, dirs, files in os.walk(search_path):
    if filename in files:
        file_path = os.path.join(root, filename)
        break
    df = pd.read_excel(file_path)

In [3]:
!ls -l

total 8
drwx------ 5 root root 4096 Feb 13 17:44 drive
drwxr-xr-x 1 root root 4096 Feb 12 14:22 sample_data


In [4]:
df=pd.read_csv(file_path)

#Load the Data and import necessary Libraries


In [5]:
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [6]:
df.tail()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1
9999,10000,15628319,Walker,792,France,Female,28,4,130142.79,1,1,0,38190.78,0


In [7]:
df.dtypes

RowNumber            int64
CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

#Data Cleaning and Filtering

In [8]:
df.drop(['RowNumber','CustomerId','Surname'],axis='columns',inplace=True)

In [9]:
df.dtypes

CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [10]:
def print_unique_col_values(df):
       for column in df:
            if df[column].dtypes=='object':
                print(f'{column}: {df[column].unique()}')

In [11]:
print_unique_col_values(df)

Geography: ['France' 'Spain' 'Germany']
Gender: ['Female' 'Male']


In [12]:
df['Gender'].replace({'Female':1,'Male':0},inplace=True)
df.Gender.unique()

array([1, 0])

In [13]:
df1 = pd.get_dummies(data=df, columns=['Geography'])
df1.columns

Index(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited',
       'Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype='object')

In [14]:
df1.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,1,42,2,0.00,1,1,1,101348.88,1,1,0,0
1,608,1,41,1,83807.86,1,0,1,112542.58,0,0,0,1
2,502,1,42,8,159660.80,3,1,0,113931.57,1,1,0,0
3,699,1,39,1,0.00,2,0,0,93826.63,0,1,0,0
4,850,1,43,2,125510.82,1,1,1,79084.10,0,0,0,1


#Use MinMax Scaler to convert remaing values between 0 and 1

In [15]:
cols_to_scale = ['CreditScore','Age','Tenure','Balance','NumOfProducts','EstimatedSalary']

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df1[cols_to_scale] = scaler.fit_transform(df1[cols_to_scale])

In [16]:
for col in df1:
    print(f'{col}: {df1[col].unique()}')

CreditScore: [0.538 0.516 0.304 0.698 1.    0.59  0.944 0.052 0.302 0.668 0.356 0.294
 0.252 0.398 0.57  0.532 0.606 0.474 0.752 0.764 0.572 0.32  0.638 0.992
 0.454 0.812 0.442 0.448 0.122 0.482 0.366 0.406 0.34  0.744 0.25  0.28
 0.908 0.464 0.244 0.23  0.412 0.968 0.62  0.852 0.958 0.574 0.4   0.696
 0.47  0.876 0.61  0.502 0.612 0.75  0.322 0.528 0.784 0.674 0.41  0.506
 0.802 0.462 0.77  0.622 0.65  0.776 0.926 0.614 0.508 0.338 0.628 0.656
 0.814 0.132 0.63  0.854 0.386 0.312 0.286 0.604 0.8   0.758 0.592 0.594
 0.916 0.348 0.838 0.76  0.33  0.846 0.928 0.72  0.126 0.546 0.64  0.544
 0.87  0.51  0.258 0.67  0.376 0.424 0.742 0.556 0.636 0.956 0.648 0.55
 0.164 0.84  0.816 0.89  0.672 0.878 0.478 0.222 0.468 0.458 0.626 0.664
 0.886 0.682 0.27  0.6   0.808 0.37  0.732 0.378 0.712 0.472 0.562 0.734
 0.9   0.666 0.708 0.53  0.634 0.268 0.26  0.456 0.324 0.512 0.494 0.856
 0.328 0.35  0.73  0.46  0.914 0.342 0.818 0.332 0.722 0.536 0.586 0.642
 0.678 0.54  0.652 0.444 0.69  0.484 0.4

In [51]:
X = df1.drop('Exited',axis='columns')
y = df1['Exited']

In [52]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=5)

In [53]:
X_train.shape

(8000, 12)

In [54]:
len(X_train.columns)

12

In [55]:
X_test.shape

(2000, 12)

#Model Implementation

In [57]:
input=keras.Input(shape=(12,))
dense=keras.layers.Dense(5,activation='relu')(input)
output=keras.layers.Dense(1,'sigmoid')(dense)

model=keras.Model(inputs=input,outputs=output)

In [58]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [59]:
model.fit(X_train,y_train,epochs=20)

Epoch 1/20
250/250 [==============================] - 2s 3ms/step - loss: 0.6586 - accuracy: 0.6499
Epoch 2/20
250/250 [==============================] - 1s 3ms/step - loss: 0.5063 - accuracy: 0.7960
Epoch 3/20
250/250 [==============================] - 1s 3ms/step - loss: 0.4789 - accuracy: 0.7962
Epoch 4/20
250/250 [==============================] - 1s 2ms/step - loss: 0.4680 - accuracy: 0.8010
Epoch 5/20
250/250 [==============================] - 1s 2ms/step - loss: 0.4614 - accuracy: 0.8046
Epoch 6/20
250/250 [==============================] - 1s 2ms/step - loss: 0.4564 - accuracy: 0.8041
Epoch 7/20
250/250 [==============================] - 1s 2ms/step - loss: 0.4519 - accuracy: 0.8059
Epoch 8/20
250/250 [==============================] - 1s 2ms/step - loss: 0.4485 - accuracy: 0.8090
Epoch 9/20
250/250 [==============================] - 1s 2ms/step - loss: 0.4448 - accuracy: 0.8096
Epoch 10/20
250/250 [==============================] - 1s 2ms/step - loss: 0.4419 - accuracy: 0.8104

In [60]:
model.evaluate(X_test,y_test)

63/63 [==============================] - 0s 2ms/step - loss: 0.4285 - accuracy: 0.8130


[0.42846766114234924, 0.8130000233650208]

In [61]:
y_test[:5]

7054    0
442     0
3954    0
2288    0
3196    0
Name: Exited, dtype: int64

In [62]:
y_p=model.predict(X_test)
y_p[:5]

63/63 [==============================] - 0s 1ms/step


array([[0.12267433],
       [0.08370335],
       [0.08632743],
       [0.07659466],
       [0.15047564]], dtype=float32)

In [63]:
y_pred = []
for element in y_p:
    if element > 0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)
y_pred[:5]

[0, 0, 0, 0, 0]